In [1]:
# This fetches the current poems from a private Drupal site accessed via xmlrpc

import xmlrpcsettings # this holds a few private settings
import xmlrpc.client
import re
import json
import urllib.request

assert xmlrpcsettings.ursite
assert xmlrpcsettings.inbookurl

proxy = xmlrpc.client.ServerProxy(xmlrpcsettings.ursite)


open_url = urllib.request.urlopen(xmlrpcsettings.inbookurl)
open_url = urllib.request.urlopen(xmlrpcsettings.inbookurl)
x = open_url.read().decode('utf-8')
nds = json.loads  ( x)

items = nds['Items']
inbook = []
for i in items:
    inbook.append( proxy.node.retrieve(i['node']['nid']) )
assert len(inbook)==30


In [39]:
def clean_trans(s):
    s = s.replace('--',u'—')
    s = s.replace(' -o- ','-o-')
    s = s.replace('\t','    ')# -o- ','-o-')
    return u''+s

def cap_trans(s):
    if s[0]==u'ḥ':
        s=u'Ḥ'+s[1:]
    return u''+s


In [59]:
import datetime
import os
import codecs

def get_ur_title(n):
    return n['field_title_urdu']['und'][0]['value']


def generate_rst(n,poem_id,nodes): 
    '''
    generates RestructuredText string from Drupal node
    n = drupal node
    poem_id = id in book, starting at 1
    
    Model:
      
    .. title: test post
    .. slug: test-post
    .. date: 2014-09-01 03:10:12 UTC
    .. tags: 
    .. link: 
    .. description: 
    .. type: text

    '''
#    n=inbook[25]
    def get_title_trans(n):
        return u''+cap_trans(clean_trans(n['field_title_transliterated']['und'][0]['value']))
    text_trans = clean_trans(n['field_text_transliterated']['und'][0]['value'])
    
    text_trans_lines = text_trans.split('\n')
    text_trans_lines = ['| '+s for s in text_trans_lines]
    trans_title = get_title_trans(n)
    text_transliterated_rst = '\n'.join(text_trans_lines)
    
    admonition = """
.. admonition:: I Too Have Some Dreams: N. M. Rashed and Modernism in Urdu Poetry

  A translation of this Urdu poem by N. M. Rashed as well as this transliteration appears in the
  appendix of *I Too Have Some Dreams*. The transliteration is intended for
  people who can understand Urdu/Hindi or related languages. I hope to soon 
  add performances of these poems as well. 
  
  .. link_figure:: /itoohavesomedreams/
        :title: I Too Have Some Dreams Resource Page
        :class: link-figure
        :image_url: /galleries/i2havesomedreams/i2havesomedreams-small.jpg
        
"""

    utc_datetime = datetime.datetime.utcnow()
    slug = 'itoohavesomedreams/poem_'+str(poem_id)    
    time_string = utc_datetime.strftime("%Y-%m-%d %H:%M:%S")+" UTC"
    
    rst_text = u'.. title: %s\n'       % (u'§'+str(poem_id)+'. '+trans_title)
    rst_text+= u'.. slug: %s\n'        % slug
    
    rst_text+= u'.. date: %s\n'        % time_string
    rst_text+= u'.. tags: %s\n'        % 'poem itoohavesomedreams rashid'
    rst_text+= u'.. link: %s\n'        % '' # what is this?
    rst_text+= u'.. description: %s\n' % ('transliterated version of "'+trans_title+'"')
    rst_text+= u'.. type: text'+'\n'
    rst_text+= u'\n'
    rst_text+= u'\n\n'    
    rst_text+= text_transliterated_rst
    rst_text+= u'\n\n'
    add_later=u''
    if poem_id>1:
        rst_text+=u'|left arrow link|_\n'
        next_poem = nodes[poem_id-2]
        prev_title = get_title_trans(nodes[poem_id-2])
        add_later+=u"\n.. |left arrow link| replace:: :emoji:`arrow_left` §{poem_id}. {poem_title} ".format(
            poem_id=poem_id-1,
            poem_title=prev_title#u''+get_title_trans(nodes[poem_id-2]) # could adjust -1 in def
        )
        add_later+='\n.. _left arrow link: /itoohavesomedreams/poem_{poem_id}\n'.format(poem_id=poem_id-1)
    if poem_id<len(nodes):
        rst_text+='\n|right arrow link|_\n'
        add_later+=u"\n.. |right arrow link| replace::  §{poem_id}. {poem_title} :emoji:`arrow_right` ".format(
            poem_id=poem_id+1,
            poem_title=get_title_trans(nodes[poem_id])#prev_title#u''+get_title_trans(nodes[poem_id-2]) # could adjust -1 in def
        )

        add_later+='\n.. _right arrow link: /itoohavesomedreams/poem_'+str(poem_id+1)+'\n'
    rst_text+='\n\n'+add_later+admonition;
#    rst_text+= u'\n\n\u2403\n'#\u2403
#    print rst_text

    assert os.path.isdir('../itoohavesomedreams')
    filename_en = '../'+slug+'.rst'
    with codecs.open(filename_en,'w','utf-8') as f: # physical location does not matter though for slug
        f.write(rst_text)

        
def generate_ur_rst(n,poem_id,nodes): 
    '''
    generates RestructuredText string from Drupal node
    n = drupal node
    poem_id = id in book, starting at 1
    
    Model:
      
    .. title: test post
    .. slug: test-post
    .. date: 2014-09-01 03:10:12 UTC
    .. tags: 
    .. link: 
    .. description: 
    .. type: text

    '''
#    n=inbook[25]
    text_ur =n['field_text_urdu']['und'][0]['value']
    text_ur_lines = text_ur.split('\n')
    text_ur_lines = ['| '+s for s in text_ur_lines]
    text_ur_rst = '\n'.join(text_ur_lines)
    trans_title = cap_trans(clean_trans(n['field_title_transliterated']['und'][0]['value']))    
    ur_title = get_ur_title(n)#n['field_title_urdu']['und'][0]['value']
    utc_datetime = datetime.datetime.utcnow()
    slug = 'itoohavesomedreams/poem_'+str(poem_id)    
    time_string = utc_datetime.strftime("%Y-%m-%d %H:%M:%S")+" UTC"
    
    rst_text = u'.. title: %s\n'       % (u'§'+str(poem_id)+u'ـ '+ur_title)
    rst_text+= u'.. slug: %s\n'        % slug
    
    rst_text+= u'.. date: %s\n'        % time_string
    rst_text+= u'.. tags: %s\n'        % 'poem itoohavesomedreams rashid'
    rst_text+= u'.. link: %s\n'        % '' # what is this?
    rst_text+= u'.. description: %s\n' % ('Urdu version of "'+trans_title+'"')
    rst_text+= u'.. type: text'+'\n'
    rst_text+= u'\n'
    rst_text+= u'\n\n'    
    rst_text+= text_ur_rst+'\n\n'
#    rst_text+= u'\n\n\u2403\n'#\u2403
 #   print rst_text
    add_later=u''
    admonition=u"""
.. admonition:: I Too Have Some Dreams: N. M. Rashed and Modernism in Urdu Poetry

  یہ ن م راشد کی نظم ہے ـ اس کا انگریزی ترجمہ اور ٹرانزلٹریشن میری کتاب
  کے ضمیمہ میں مل سکتا ہےـ اردو
  پڑھنے والوں کے لئے یہ پیج پیش کیا گیا ہےـ نستعلیق میں
  دکھانے کے لئے 
  `جمیل نوری نستعلیق فانٹ`_  انسٹال کیجئے.


  .. link_figure:: /itoohavesomedreams/
        :title: I Too Have Some Dreams Resource Page
        :class: link-figure
        :image_url: /galleries/i2havesomedreams/i2havesomedreams-small.jpg
        
.. _جمیل نوری نستعلیق فانٹ: http://ur.lmgtfy.com/?q=Jameel+Noori+nastaleeq
 

"""
    if poem_id>1:
      rst_text+=u'\n|right arrow link|_\n'
      next_poem = nodes[poem_id-2]
      prev_title = get_ur_title(nodes[poem_id-2])
      add_later+=u"\n.. |right arrow link| replace:: :emoji:`arrow_right` §{poem_id}. {poem_title}  ".format(
          poem_id=poem_id-1,
          poem_title=prev_title#u''+get_title_trans(nodes[poem_id-2]) # could adjust -1 in def
      )
      add_later+=u'\n.. _right arrow link: /ur/itoohavesomedreams/poem_{poem_id}\n'.format(poem_id=poem_id-1)
    if poem_id<len(nodes):
      rst_text+=u'\n|left arrow link|_\n'
      add_later+=u"\n.. |left arrow link| replace::   §{poem_id}. {poem_title} :emoji:`arrow_left` ".format(
          poem_id=poem_id+1,
          poem_title=get_ur_title(nodes[poem_id])#prev_title#u''+get_title_trans(nodes[poem_id-2]) # could adjust -1 in def
      )
      add_later+=u'\n.. _left arrow link: /ur/itoohavesomedreams/poem_'+str(poem_id+1)+'\n'
  
    rst_text+='\n\n'+add_later+admonition;
    
        
    assert os.path.isdir('../itoohavesomedreams')
    filename_en = '../'+slug+'.ur'+'.rst' # format is : title.languageid.rst
    
    with codecs.open(filename_en,'w','utf-8') as f: # physical location does not matter though for slug
        f.write(rst_text)
        
         
    
import sys
sys.path.append('./graphparser')
import graphparser as gp
import importlib
importlib.reload(gp)

dp = gp.GraphParser('graphparser/settings/devanagari.yaml')



In [71]:
numbers={'hi': '०१२३४५६७८९',
         'ur': '۰۱۲۳۴۵۶۷۸۹',
         'pa': '੦੧੨੩੪੫੬੭੮੯'}
def num_as(x,lang='en'):
    if lang=='en': return str(x)
    num_str = numbers[lang]
    out = ''
    for c in str(x):
        o = ord(c) - ord('0')
        out +=num_str[o]
    return out
num_as(120,lang='ur')


'۱۲۰'

In [91]:
def clean_hi_trans(s):
#    s = s.replace('--',u'—')
    s = s.replace(' -o- ','-o-')
    s = s.replace('\t','    ')# -o- ','-o-')
    s = s.replace('^','')
#    s= s.replace('-e ','-e-')
    return u''+s

def index_link(n,n_id,lang='en'):
    trans_title = cap_trans(clean_trans(n['field_title_transliterated']['und'][0]['value']))
    if lang=='en':
        return '  `§'+num_as(n_id,'en')+u'. '+trans_title+u' <poem_'+str(n_id)+u'/>'+'`_'+'\n'
    elif lang=='hi':
        hi_title = get_hi_title(n)
        return '  `§'+num_as(n_id,'hi')+u'. '+hi_title+u' <poem_'+str(n_id)+u'/>'+'`_'+'\n'
    elif lang=='ur':
        ur_title = get_ur_title(n)
        return '  `'+num_as(n_id,'ur')+u'۔ '+ur_title+u' <poem_'+str(n_id)+u'/>'+'`_'+'\n'
    elif lang=='pa':
        hi_title = get_hi_title(n)
        pa_title = punjabify_hi(hi_title)
        return '  `§'+num_as(n_id,'pa')+u'. '+pa_title+u' <poem_'+str(n_id)+u'/>'+'`_'+'\n'



def get_hi_title(n):
        t = n['field_title_markup']['und'][0]['value']
        t = t.lower()
        return dp.parse(t).output

def get_pa_title(n):
        x = get_hi_title(n)
        return punjabify_hi(x)
    
def get_hi_text(n):
    t = n['body']['und'][0]['value']
    t = t.lower()
    lines = t.split('\n')
    hi_lines = [dp.parse(clean_hi_trans(l)).output for l in lines]
    hi_lines = ['| '+s for s in hi_lines]
    out = '\n'.join(hi_lines)
#    out = ['| '+s for s in out]
    return out
    

def generate_hi_rst(n,poem_id,nodes): 
    '''
    generates RestructuredText string from Drupal node
    n = drupal node
    poem_id = id in book, starting at 1
    
    Model:
      
    .. title: test post
    .. slug: test-post
    .. date: 2014-09-01 03:10:12 UTC
    .. tags: 
    .. link: 
    .. description: 
    .. type: text

    '''
#    n=inbook[25]
    def get_ur_title(n):
        return n['field_title_urdu']['und'][0]['value']
    
    def get_hi_title(n):
        t = n['field_title_markup']['und'][0]['value']
        t = t.lower()
        return dp.parse(t).output
        
    
#  #  text_hi =n['field_text_urdu']['und'][0]['value']
#   text_ur_lines = text_ur.split('\n')
#   text_ur_lines = ['| '+s for s in text_ur_lines]
    text_ur_rst = get_hi_text(n)
    
    trans_title = cap_trans(clean_trans(n['field_title_transliterated']['und'][0]['value']))    
    ur_title = get_hi_title(n)#n['field_title_urdu']['und'][0]['value']
    utc_datetime = datetime.datetime.utcnow()
    slug = 'itoohavesomedreams/poem_'+str(poem_id)    
    time_string = utc_datetime.strftime("%Y-%m-%d %H:%M:%S")+" UTC"
    
    rst_text = u'.. title: %s\n'       % (u'§'+str(poem_id)+u'ـ '+ur_title)
    rst_text+= u'.. slug: %s\n'        % slug
    
    rst_text+= u'.. date: %s\n'        % time_string
    rst_text+= u'.. tags: %s\n'        % 'poem itoohavesomedreams rashid'
    rst_text+= u'.. link: %s\n'        % '' # what is this?
    rst_text+= u'.. description: %s\n' % ('Devanagari version of "'+trans_title+'"')
    rst_text+= u'.. type: text'+'\n'
    rst_text+= u'\n'
    rst_text+= u'\n\n'    
    rst_text+= text_ur_rst+'\n\n'
    add_later=u''
    admonition=u"""
.. admonition:: I Too Have Some Dreams: N. M. Rashed and Modernism in Urdu Poetry


  .. link_figure:: /itoohavesomedreams/
        :title: I Too Have Some Dreams Resource Page
        :class: link-figure
        :image_url: /galleries/i2havesomedreams/i2havesomedreams-small.jpg
        
.. _جمیل نوری نستعلیق فانٹ: http://ur.lmgtfy.com/?q=Jameel+Noori+nastaleeq
 

""" 
    if poem_id>1:
        rst_text+=u'|left arrow link|_\n'
        next_poem = nodes[poem_id-2]
        prev_title = get_hi_title(nodes[poem_id-2])
        add_later+=u"\n.. |left arrow link| replace:: :emoji:`arrow_left` §{poem_id}. {poem_title} ".format(
            poem_id=poem_id-1,
            poem_title=prev_title#u''+get_title_trans(nodes[poem_id-2]) # could adjust -1 in def
        )
        add_later+='\n.. _left arrow link: /hi/itoohavesomedreams/poem_{poem_id}\n'.format(poem_id=poem_id-1)
    if poem_id<len(nodes):
        rst_text+='\n|right arrow link|_\n'
        add_later+=u"\n.. |right arrow link| replace::  §{poem_id}. {poem_title} :emoji:`arrow_right` ".format(
            poem_id=poem_id+1,
            poem_title=get_hi_title(nodes[poem_id])#prev_title#u''+get_title_trans(nodes[poem_id-2]) # could adjust -1 in def
        )

        add_later+='\n.. _right arrow link: /hi/itoohavesomedreams/poem_'+str(poem_id+1)+'\n'
    rst_text+='\n\n'+add_later+admonition;
    assert os.path.isdir('../itoohavesomedreams')
    filename_en = '../'+slug+'.hi'+'.rst' # format is : title.languageid.rst
    
    with codecs.open(filename_en,'w','utf-8') as f: # physical location does not matter though for slug
        f.write(rst_text)
        

In [92]:
def punjabify_hi(x):
    out=u''
    for c in x:
        if ord(c) in range(0x900, 0x97F):
            out+= chr(ord(c) + 0xA00 - 0X900)  
        else:
            out += c
    return out

print(punjabify_hi(u'है'))

ਹੈ


In [93]:
punjabify_hi(x)
def generate_pa_rst(poem_id): 
    slug = 'itoohavesomedreams/poem_'+str(poem_id) 

    filename_hi = '../'+slug+'.hi'+'.rst' # format is : title.languageid.rst
    with codecs.open(filename_hi,'r','utf-8') as f: # physical location does not matter though for slug
        text=f.read()

    new_text=punjabify_hi(text)

    filename_pa = '../'+slug+'.pa'+'.rst'
    with codecs.open(filename_pa,'w','utf-8') as f: # physical location does not matter though for slug
        f.write(new_text)


In [97]:


for n_id, n in enumerate(inbook):
    generate_rst(n,n_id+1,inbook)
    generate_ur_rst(n,n_id+1,inbook)
    generate_hi_rst(n,n_id+1,inbook)#,inbook)
    generate_pa_rst(n_id+1)#,inbook)
    index_link(n,n_id+1)
    

In [95]:
index_link(inbook[1],1,lang='hi')

'  `§१. एक दिन—लारिंस बाग़ में (एक कैफ़ियत) <poem_1/>`_\n'

In [96]:
for lang in ['hi','ur','pa']:
    for n_id, n in enumerate(inbook):
        print(index_link(n,n_id+1,lang=lang))

  `§१. बादल (सानेट) <poem_1/>`_

  `§२. एक दिन—लारिंस बाग़ में (एक कैफ़ियत) <poem_2/>`_

  `§३. सितारे (सानेट) <poem_3/>`_

  `§४. वादी-ए पिंहाँ <poem_4/>`_

  `§५. गुनाह और मुहब्बत <poem_5/>`_

  `§६. मुकाफ़ात <poem_6/>`_

  `§७. हुज़्न-ए इंसान (अफ़्लातूनी इश्क़ पर एक तंज़) <poem_7/>`_

  `§८. इत्तिफ़ाक़ात <poem_8/>`_

  `§९. शायर-ए दर-मांदा <poem_9/>`_

  `§१०. इंतिक़ाम <poem_10/>`_

  `§११. हमह ऊस्त <poem_11/>`_

  `§१२. तेल के सौदागर <poem_12/>`_

  `§१३. मंन ‐ओ‐ सलवा <poem_13/>`_

  `§१४. तमाशागा-ए लाला-ज़ार <poem_14/>`_

  `§१५. नमरूद की ख़ुदाई <poem_15/>`_

  `§१६. वो हर्फ़-ए तंहा (जिसे तमन्ना-ए वस्ल-ए माना) <poem_16/>`_

  `§१७. ज़िंदगी से डरते हो? <poem_17/>`_

  `§१८. दिल, मिरे सहरा-नवर्द-ए पीर दिल <poem_18/>`_

  `§१९. एक और शहर <poem_19/>`_

  `§२०. रेग-ए दीरूज़ <poem_20/>`_

  `§२१. ज़माना ख़ुदा है <poem_21/>`_

  `§२२. अफ़्साना-ए शहर <poem_22/>`_

  `§२३. ये ख़ला पुर न हुआ <poem_23/>`_

  `§२४. तलब के तले <poem_24/>`_

  `§२५. गुमाँ का मुम्किन—जो तू है मैं हूँ! <poem_25/>